In [5]:
%load_ext sql
%sql sqlite:///

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: @'

Problem Set 2
=======


### Instructions / Notes:

**_Read these carefully_**

* This problem set _does not_ come with a dataset to load; instead, make your own instances of tables, either as solutions to the problems or for testing solutions to the problems.
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* **See Piazza for submission instructions**
* _Have fun!_

Problem 1
---------

**_[15 points total]_**

For each part of this problem you will need to provide a _single_ SQL query which will check whether a certain condition holds on a specific instance of a relation, in the following way: **your query should return an empty result if and only if the condition holds on the instance.**  (If the condition _doesn't hold_, your query should return something non-empty, but it doesn't matter what this is).

Note our language here: the conditions that we specify cannot be proved to hold **in general** without knowing the externally-defined functional dependencies; so what we mean is, _check whether they **could** hold in general for the relation, given any specific set of tuples_.

You may assume that there will be no `NULL` values in the tables, **and you may assume that the relations are _sets_ rather than multisets**, but otherwise your query should work for general instances.  We define the schemas of the tables used below for convenience, but in this problem you will need to construct your own test tables if you wish to use them to check your answers!

In [7]:
%%sql
DROP TABLE IF EXISTS R; CREATE TABLE R (W INT, X INT, Y INT, Z INT);
DROP TABLE IF EXISTS Cat; CREATE TABLE Cat(cat_name TEXT, breed TEXT, owner_name TEXT);
DROP TABLE IF EXISTS Owner; CREATE TABLE Owner(owner_name TEXT, ssn INT, hometown TEXT);
DROP TABLE IF EXISTS S; CREATE TABLE S (A INT, B INT, C INT, D INT, E INT);

 * sqlite:///
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### Part (a)

**_[5 points]_**

$\{X, Y\}$ is a **superkey** for a relation $R(W,X,Y,Z)$.

In [9]:
%%sql
select X, Y
from R
group by X, Y
having count(*) > 1;

 * sqlite:///
Done.


X,Y


### Part (b)

**_[5 points]_**

The individual attributes of a relation $R(W,X,Y,Z)$ are each keys.

In [11]:
%%sql
select *
from (
         select W as Duplicate_Key
         from R
         group by W
         having count(*) > 1
         union
         select X as Duplicate_Key
         from R
         group by X
         having count(*) > 1
         union
         select Y as Duplicate_Key
         from R
         group by Y
         having count(*) > 1
         union
         select Z as Duplicate_Key
         from R
         group by Z
         having count(*) > 1
     );

 * sqlite:///
Done.


Duplicate_Key


### Part (c)

**_[5 points]_**

A **multivalued dependency (MVD)** is defined as follows: let $R$ be a schema i.e. a set of attributes, and consider two _sets of attributes_ $A\subseteq R$ and $B\subseteq R$.  We say that a multivalued dependency (MVD), written:

$A\twoheadrightarrow B$

**holds on $R$** if whenever there are two tuples $t_1,t_2$ such that $t_1[A] = t_2[A]$, there also exists a third tuple $t_3$ such that:

* $t_3[A] = t_1[A] = t_2[A]$
* $t_3[B] = t_1[B]$
* $t_3[R\setminus B] = t_2[R\setminus B]$

Note that $R\setminus B$ is all the attributes in $R$ that are not in $B$, and that $t_3$ need not be distinct from $t_1$ or $t_2$.  Note especially that an MVD holds on an entire _relation_, meaning that any two tuples (in any order) in the relation should satisfy the above conditions if the MVD holds.  **See the end of the lecture 7 slides for more on MVDs!**


In this problem, write your query to check if the MVD $\{B\}\twoheadrightarrow \{D,E\}$ holds for a relation $S(A, B, C, D, E)$.

In [12]:
%%sql 
select *
from (
         select S.A, S.B, S.C, S.D, S.E
         from S,
              (
                  select A multi_a, B multi_b, C multi_c
                  from (
                           select A, B, C
                           from S,
                                (
                                    select B multi_b
                                    from S
                                    group by B
                                    having count(*) > 2
                                )
                           where S.B = multi_b
                       )
                  group by A, B, C
                  having count(*) > 1
              )
         where S.A = multi_a
           and S.B = multi_b
           and S.C = multi_c
     )
where exists
          (
              select S.A, S.B, S.C, S.D, S.E
              from S,
                   (
                       select B multi_b, D multi_d, E multi_e
                       from (
                                select B, D, E
                                from S,
                                     (
                                         select B multi_b
                                         from S
                                         group by B
                                         having count(*) > 2
                                     )
                                where S.B = multi_b
                            )
                       group by B, D, E
                       having count(*) > 1
                   )
              where S.D = multi_d
                and S.E = multi_e
                and S.B = multi_b
          );

 * sqlite:///
Done.


A,B,C,D,E


Problem 2
---------

**_[20 points total]_**

### Part (a)

**_[10 points]_**

Consider a relation $T(V,W,X,Y,Z)$.  Provide _two different sets_ of functional dependencies, `F_1` and `F_2`, such that each one results in $T$ having the **largest number of distinct keys** $T$ could possibly have.

Store your lists of FDs as python lists having elements that are _pairs of sets_; for example to set `F_1` as the set consisting of two FDs, $\{V,W\}\rightarrow\{X,Y\}$ and $\{W\}\rightarrow\{X\}$:

```python
F_1 = [(set(['V','W']), set(['X','Y'])), (set(['W']), set(['X']))]
```

*Note: the above is not necessarily one of the FDs- just an example of the syntax!

*Hint: You may use any of the functions from the lecture activities if they seem helpful!  However your final answer should not involve these functions directly, nor are they necessary for this problem

In [ ]:
F_1 = [
    (set(['V']), set(['W','X','Y','Z'])),
    (set(['W']), set(['V'])),
    (set(['X']), set(['V'])),
    (set(['Y']), set(['V'])),
    (set(['Z']), set(['V'])),
]

F_2 = [
    (set(['W']), set(['V','X','Y','Z'])),
    (set(['V']), set(['W'])),
    (set(['X']), set(['W'])),
    (set(['Y']), set(['W'])),
    (set(['Z']), set(['W'])),
]

### Part (b)

**_[10 points]_**

Consider a schema $T(A_1,...,A_n)$ which has FDs $\{A_i,A_{i+1}\}\rightarrow\{A_{i+2}\}$ for $i=1,...,n-2$.  Create an instance of $T$, for $n=4$, for which these FDs hold, and no other ones do- i.e. **all other FDs are violated.**

Use a series of `INSERT` statements below to populate the table `T`:

In [ ]:
%%sql
DROP TABLE IF EXISTS T; 
CREATE TABLE T (A int, B int, C int, D int);
insert into T
values (1, 2, 5, 4),
       (1, 3, 5, 6),
       (2, 2, 5, 4),
       (2, 3, 5, 6);

Problem 3
---------

**_[30 points total]_**

Consider a relation $R(X,Y,Z)$.  In each part of this problem you will be given a condition, and you need to create the following three instances of $R$ (as tables in SQL):

1. An instance $A$
2. An instance $B$ which differs from $A$ only in that it has one **_fewer_** row. Any row in B should also be there in A.
3. An instance $C$ which differs from $A$ only in that it has one **_additional_** row. Apart from the additional row, all the rows of C are same as A.

### Part (a)

**_[10 points]_**

Create $A$, $B$ and $C$ such that each individual attribute is a key for $A$, but none of the individual attributes is a key for $B$ or $C$.  If you believe that $B$ and/or $C$ cannot be created, provide them as an empty table.

In [ ]:
%%sql
DROP TABLE IF EXISTS A;
DROP TABLE IF EXISTS B;
DROP TABLE IF EXISTS C;
create table A ( X int, Y int, Z int );
create table B ( X int, Y int, Z int );
create table C ( X int, Y int, Z int );

insert into A
values 
       (1, 4, 7),
       (2, 5, 8);

insert into C
values 
       (1, 4, 7),
       (2, 5, 8),
       (1, 4, 8);

### Part (b)

**_[10 points]_**

Create $A$, $B$ and $C$ such that ONLY the functional dependencies $\{Z\} \rightarrow \{Y\}$ and $\{X,Z\} \rightarrow \{Y\}$ hold on $B$, ONLY the functional dependency $\{X,Z\} \rightarrow \{Y\}$ holds on $A$ and NO functional dependencies hold on $C$. If you believe $B$ and/or $C$ cannot be created, provide them as an empty table.



In [ ]:
%%sql
DROP TABLE IF EXISTS A;
DROP TABLE IF EXISTS B;
DROP TABLE IF EXISTS C;
create table A ( X int, Y int, Z int );
create table B ( X int, Y int, Z int );
create table C ( X int, Y int, Z int );

insert into A
values (2, 4, 7),
       (2, 4, 8),
       (3, 4, 8);

insert into B
values (2, 4, 7),
       (2, 4, 8);

insert into C
values (2, 4, 7),
       (2, 4, 8),
       (3, 4, 8),
       (2, 5, 7);


### Part (c)

**_[10 points]_**

Create $A$, $B$ and $C$ such that the MVD $Z\twoheadrightarrow X$ holds in $A$, but not in $B$ or $C$.  If you believe that $B$ and/or $C$ cannot be created, provide them as an empty table.



In [ ]:
%%sql
DROP TABLE IF EXISTS A;
DROP TABLE IF EXISTS B;
DROP TABLE IF EXISTS C;
create table A ( X int, Y int, Z int );
create table B ( X int, Y int, Z int );
create table C ( X int, Y int, Z int );

insert into A
values (1, 4, 7),
       (2, 5, 7),
       (2, 4, 7),
       (1, 5, 7);

insert into B
values (1, 4, 7),
       (2, 5, 7),
       (2, 4, 7);

insert into C
values (1, 4, 7),
       (2, 5, 7),
       (2, 4, 7),
       (1, 5, 7),
       (3, 4, 7);